In [ ]:
import numpy as np
import networkx as nx
import datetime
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#zadanie 1
#Funkcja działająca na macierzach zaimplementowanych w numpy
def gauss_solve(A):
    inversions=[i for i in range(len(A))]
    for i in range(len(A)):
        max_i=i
        for j in range(i,len(A)):
            if abs(A[max_i][i])<abs(A[j][i]):
                max_i = j
        best_row=np.copy(A[max_i])
        A[[i,max_i]]=A[[max_i,i]]#Zamieniam wiersze miejscami, tak, aby na górze było zawsze to o największej wartości bezwzględnej
        inversions[i],inversions[max_i]=inversions[max_i],inversions[i]
        for j in range(i+1,len(A)):
            first_non_zero=A[j][i]
            A[j]-=best_row*first_non_zero/best_row[i]
    #uzyskałem macierz trójkątną górną
    #teraz z niej uzyskuję wyniki
    for i in range(len(A)-1,-1,-1):
        for j in range(i-1,-1,-1):
            last_non_zero=A[j][i]
            A[j]-=A[i]*last_non_zero/A[i][i]
    for i in range(len(A)):
        A[i]/=A[i][i]
    result=A[:,-1]
    result1=np.zeros((len(A),))
    for i,r in enumerate(inversions):
        result1[r]=result[i]
    return result

In [ ]:
M=np.random.random((30,31))

In [ ]:
M

In [ ]:
result=gauss_solve(np.copy(M))
print(result)

In [ ]:
M[:,-1]#B

In [ ]:
np.dot(np.copy(M)[:,:30],result.reshape(30,1))#Mnożę macierz współczynników, przez wektor wyniku,
#aby zweryfikować go. Wychodzi równy ostatniej kolumnie macierzy

In [ ]:
sizes=[600+200*i for i in range(10)]

In [ ]:
own_times=[]
lib_times=[]
for s in sizes:
    M=np.random.random((s,s+1))
    start=datetime.datetime.now()
    A=gauss_solve(M)
    stop=datetime.datetime.now()
    own_times.append((stop-start).microseconds//1000)#datetime ma dokładność do nanosekund, więc ucinam zera na końcu
    start=datetime.datetime.now()
    A=np.linalg.solve(M[:,:s],M[:,-1])
    stop=datetime.datetime.now()
    lib_times.append((stop-start).microseconds//1000)

In [ ]:
plt.plot(sizes,own_times,color='green')
plt.plot(sizes,lib_times,color='red')
plt.title("Czas działania algorytmu dla danych danych")
plt.xlabel("Matrix size")
plt.ylabel("time [ns]")
plt.show()

In [ ]:
#zadanie 2
#faktoryzacja macierzy kwadratowej metodą Doolittle'a
def factorize(M):
    size=M.shape[0]
    L=np.identity(size)
    U=np.zeros((size,size))
    for i in range(size):
        for j in range(i,size):
            sub_sum=0
            for k in range(i):
                sub_sum+=L[i][k]*U[k][j]
            U[i][j]=M[i][j]-sub_sum
        for j in range(i+1,size):
            sub_sum=0
            for k in range(i):
                sub_sum+=L[j][k]*U[k][i]
            L[j][i]=(M[j][i]-sub_sum)/U[i][i]
    return L,U

In [ ]:
M=np.array([[5,3,2],[1,2,0],[3,0,4]])
print(M)
print(factorize(M)[0])
print(factorize(M)[1])

In [ ]:
M=np.random.random((5,5))
L,U=factorize(M)
print(M-np.dot(L,U))

# Możemy uznać, że funkcja factorize działa poprawnie.
W każdym wierszu w każdej kolumnie wychodzą wartośći 0 lub bliskie 0.

### zadanie 3
a)
Plik zawierający graf musi mieć postać:<br>
v1 u1 w1<br>
v2 u2 w2<br>
v3 u3 w3<br>
.  .  .<br>
.  .  .<br>
gdzie ui i vi to wierzcholki grafu, a wi to wagi krawędzi między nimi

In [ ]:
def read_graph(name):
    L=[]
    with open(name,'r') as file:
        lines=file.readlines()
        for l in lines:
            s=l.split()
            u,v,w=int(s[0]),int(s[1]),int(s[2])
            L.append((u,v,w))
    G=nx.Graph()
    for u,v,w in L:
        G.add_edge(u-1, v-1, weight=w)
    return G

In [ ]:
def randomize_resistance(G):
    for e in G.edges():
        G[e[0]][e[1]]['weight']=np.random.random()

In [ ]:
def random_graph(n):
    G=nx.gnp_random_graph(n,np.random.randint(n,n*(n-1)//2))
    randomize_resistance(G)
    return G

In [ ]:
def cycles(G):
    #Biblioteczna funkcja cycle_basis zwraca tylko cykle rozłączne.
    #Jest ich dokładnie tyle, aby nasz układ równań był określony.
    return [[i for i in c] for c in nx.cycle_basis(G)]

In [ ]:
#R=U/I
#I=U/R
#U=R*I
def prepare_matrix(G,s,t,E):
    weights=nx.get_edge_attributes(G,'weight')
    if not (G.has_edge(s,t)or G.has_edge(t,s)):
        G.add_edge(s,t)
    n_list=[[n for n in G.neighbors(i)]for i in range(len(G))]
    edges=[ t for t in G.edges()]
    edges_dict=dict()
    n=len(n_list)
    m=len(edges)
    for i in range(m):
        edges_dict[edges[i]]=i
    c=cycles(G)
    A=np.zeros((n+len(c),n+len(c)))
    B=np.zeros((n+len(c),))
    #dodaję do układu równań równania wynikające z drugiego prawa kirchoffa
    for i in range(len(c)):
        cycle=c[i]
        cycle_edges=zip(cycle,cycle[1:]+[cycle[0]])
        for e in cycle_edges:
            norm_e=(min(e),max(e))
            if e == (s,t):
                B[i]= -E
            elif e == (t,s):
                B[i]= E
            else:
                e_index=edges_dict[norm_e]
                if e in edges_dict:
                    A[i][e_index]=weights[norm_e]
                else:
                    A[i][e_index]=-weights[norm_e]
            if e == (cycle[0],cycle[1]):
                B[i]*=-1
    #dodaję do układu równań równania wynikające z pierwszego prawa kirchoffa
    for i in range(n):
        for j in n_list[i]:
            if j>i:
                e=(i,j)
                e_index=edges_dict[e]
                A[len(c)+i,e_index]=1
            else:
                e=(j,i)
                e_index=edges_dict[e]
                A[len(c)+i,e_index]=-1
    if G.has_edge(s,t):
        G[s][t]['weight']=0
    else:
        G[t][s]['weight']=0
    return A[:m,:m],B[:m],edges_dict
            

In [ ]:
#funkcja generująca graf skierowany natężeń prądu
def to_directed(G):
    N=nx.DiGraph()
    for edge in G.edges():
        intensity=G[edge[0]][edge[1]]['weight']
        if intensity > 0:
            N.add_edge(edge[0],edge[1])
            N[edge[0]][edge[1]]['weight']=intensity
        else:
            N.add_edge(edge[1],edge[0])
            N[edge[1]][edge[0]]['weight']=-intensity
    return N
    

In [ ]:
def adjust(G,e,I):
    I_graph=G.copy(as_view=True)
    for edge in I_graph.edges():
        I_graph[edge[0]][edge[1]]['weight']=I[e[edge]]
    return to_directed(I_graph)

In [ ]:
#funkcja przygotowująca i prezentująca na podstawie wektora  rozwiązań, gotowy graf natężeń
def show(I_graph,pos):
    e_labels = dict([((n1, n2), round(d['weight'],2)) for n1, n2, d in I_graph.edges(data=True)])
    edges,weights = zip(*nx.get_edge_attributes(I_graph,'weight').items())
    fig = plt.figure(1, figsize=(25, 20))
    nx.draw(I_graph, pos,with_labels=True, node_color='b', edgelist=edges, edge_color=weights, width=10.0, edge_cmap=plt.cm.cividis)
    nx.draw_networkx_edge_labels(I_graph,pos,edge_labels=e_labels,font_color='blue')
    plt.show()

In [ ]:
#inicjalizacja danych zadania
G=random_graph(7)
s=0
t=1
E=2
pos = nx.spring_layout(G)

In [ ]:
show(G,pos)

In [ ]:
#Prezentacja grafu skierowanego natężeń i obliczenie rozwiązania
def solve_graph(G,s,t,E):
    A,B,e=prepare_matrix(G,s,t,E)#macierz wynika z układów równań
    I=np.linalg.solve(A,B)#wektor rozwiązań
    I_graph=adjust(G,e,I)
    return I_graph
solution = solve_graph(G.copy(),s,t,E)
show(solution,pos)

In [ ]:
def check_graph(G,R,s,t,E):
    #Sprawdzenie, czy dla obwodu zachodzi pierwsze prawo kirchoffa
    n_list=[[n for n in G.neighbors(i)]for i in range(len(G))]
    Nodes_sum=[0 for n in n_list]
    for edge in G.edges():
        Nodes_sum[edge[0]]-=G[edge[0]][edge[1]]['weight']
        Nodes_sum[edge[1]]+=G[edge[0]][edge[1]]['weight']
    for n in Nodes_sum:
        if abs(n) > 10**(-13):
            return False
    cycles = nx.simple_cycles(G)
    G_edges= set(G.edges)
    #Sprawdzenie, czy dla obwodu zachodzi drugie prawo kirchoffa
    for cycle in cycles:
        cycle_edges=zip(cycle,cycle[1:]+[cycle[0]])
        cycle_sum = 0
        for edge in cycle_edges:
            if edge == (s,t):
                U=E
            elif edge == (t,s):
                U=-E
            else:
                if (edge[0],edge[1]) in G_edges:
                    U=G[edge[0]][edge[1]]['weight']*R[edge[0]][edge[1]]['weight']
                else:
                    U=-G[edge[1]][edge[0]]['weight']*R[edge[0]][edge[1]]['weight']
            if edge == (cycle[-1],cycle[1]):
                U*=-1
            cycle_sum +=U
        if abs(cycle_sum) > 10**(-13):
            return cycle_sum
    return False
    

In [ ]:
check_graph(solution,G,s,t,E)#wartość false oznacza brak błędu

In [ ]:
def double_random_graph(n):
    A = random_graph(n//2)
    B = random_graph(n//2)
    x=len(A.nodes)
    for edge in B.edges:
        A.add_edge(edge[0]+x,edge[1]+x)
    A.add_edge(0,x)
    randomize_resistance(A)
    return A

In [ ]:
def grid_graph(n):
    G = nx.grid_graph(dim=[int(np.sqrt(n)),int(np.sqrt(n))+1], periodic=False)
    mapping = dict(zip(G, range(int(np.sqrt(n))*(int(np.sqrt(n))+1))))
    G = nx.relabel_nodes(G, mapping)
    #liczba wierzchołków jest bliska n
    randomize_resistance(G)
    return G

In [ ]:
def small_world_graph(n):
    G = nx.watts_strogatz_graph(n, 4, 0.2)
    randomize_resistance(G)
    return G

In [ ]:
graph_types = [random_graph,double_random_graph,grid_graph,small_world_graph]
graph_sizes = [15, 30, 60 ,120 ,200]

In [ ]:
def test():
    #AUTOMATYCZNE TESTOWANIE I WIZUALIZACJA GRAFÓW
    G=nx.cubical_graph()
    randomize_resistance(G)
    E=np.random.random()*10
    solution = solve_graph(G.copy(),0,1,E)
    pos = nx.spring_layout(G)
    show(G,pos)
    show(solution,pos)
    checking = check_graph(solution,G,0,1,E)
    if checking:
        print("BŁĄD PROGRAMU: NIEWŁAŚCIWE ROZWIĄZANIE ",checking)
    for gt in graph_types:
        for size in graph_sizes:
            G=gt(size)
            randomize_resistance(G)
            E=np.random.random()*10
            pos = nx.spring_layout(G)
            show(G,pos)
            solution = solve_graph(G.copy(),0,1,E)
            show(solution,pos)
            checking = check_graph(solution,G,0,1,E)
            if checking:
                print("BŁĄD PROGRAMU: NIEWŁAŚCIWE ROZWIĄZANIE ",checking)

In [ ]:
test()

In [ ]:
#test dla największego (200 węzłów) grafu siatki wykonywał się bardzo długo, dlatego przerwałem obliczenia
#testy dla grafów typu small world:
graph_types = [small_world_graph]
test()

na moje rozwiązanie składają się:
-skonstruowanie układu równań
-rozwiązanie go
-utworzenie na tej podstawie grafu natężeń pradu
-wizualizacja tego grafu
-przetestowanie otrzymanego grafu

w celu skonstruowania układu równań, znajduję wszystkie cykle bazowe grafu za pomocą funkcji nx.cycle_basis(). Funkcja ta znajduje drzewo rozpinające w grafie i dla każdej krawędzi nieznajdujących się w tym drzewie wyznacza cykle. cykli tych będzie dokładnie m-n
https://en.wikipedia.org/wiki/Cycle_basis
Do utworzenia układu równań mógłbym skorzystać rownież z funkcji nx.simple_cycles(). Jednakże wówczas otrzymałbym nadokreślony układ równań.
Równania otrzymywane poprzez II prawo kirchoffa dla każdego z tych cykli będą liniowo niezależne, gdyż z tej bazy cykli da się poprzez różnicę symetryczną otrzymać dowolny inny cykl w grafie.
Oprocz tego, stosując pierwsze prawo kirchoffa otrzymuję układ n liniowo niezależnych równań.
Taki układ rownań rozwiązuję za pomocą bibliotecznej funkcji np.linalg.solve()
Tworząc graf natężeń prądu, krawędzie kierowałem od wierzchołka o mniejszy numerze do tego o większym, a jeżeli natężenie prądu w rozwiązniu było ujemne, odwracalem kierunek krawędzi. W celu sprawdzenia poprawności rozwiązania, sprawdzam, czy w grafie zachodzą I i II prawo kirchoffa dla wszystkich węzłów i cykli.